In [4]:
from sql_functions import get_dataframe
import pandas as pd
import Capstone_functions as cf
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy import stats
from scipy import interpolate
from sklearn import linear_model

# Variables

In [5]:
top_XX_mechanic = 25

schema = "bgg_data"
main = "ml_boardgame_stats"
subdomain = "subdomain"
unique_subdomain = "unique_subdomain"
kickstarter = "kickstarter_unique_campaigns"
slug = "unique_slug_bgg_id"
mechanic = "mechanics"
unique_mechanics = "unique_mechanics"


#honor = "honor_clean"



## Create Dataframes for Kickstarter and Mechanic

In [6]:

df_ks = get_dataframe(f"SELECT * FROM {schema}.{kickstarter}")
df_slug = get_dataframe(f"SELECT * FROM {schema}.{slug}")
df_mech = get_dataframe(f"SELECT * FROM {schema}.{mechanic}")
df_u_mech = get_dataframe(f"SELECT * FROM {schema}.{unique_mechanics}")

#df_marketplace = cf.avg_price_from_marketplace()
#df_honor = get_dataframe(f"SELECT * FROM {schema}.{honor}")

Merge kickstarter table with bgg_ids and rename column bgg_id to id

In [7]:
df_ks = pd.merge(df_slug,df_ks,on='slug')
df_ks.rename({'bgg_id':'id'},axis=1,inplace=True)

create dataframe with all mechanics and merge it with the kickstarter dataframe

In [8]:
df_mech = pd.merge(df_mech,df_u_mech,on='mechanic_id')
df_mech = pd.merge(df_ks,df_mech,on='id')

create a list with the top XX mechanics

In [9]:
top_mechanics_list = list(df_mech.groupby(['mechanic']).count().sort_values(ascending=False,axis=0,by='id').reset_index()["mechanic"].head(top_XX_mechanic))

reduce the dataframe such that IDs are unique

In [10]:
df_mech["is_in_top_XX_mechanics"] = df_mech["mechanic"].isin(top_mechanics_list)
df_mech = df_mech[['id','is_in_top_XX_mechanics']]
df_mech = df_mech.groupby('id').sum().reset_index()

Because there are multiple mechanics for one ID, it is possible that for the same ID there are some mechanics in the top and others are not.
- create new column with True/False if ID is in top XX categories (XX will be set in the top code field: "Variables" -> "top_XX_mechanic")

In [11]:
df_mech[f"top_{top_XX_mechanic}_mechanic"] = df_mech.is_in_top_XX_mechanics > 0

In [12]:
df_mech = df_mech[['id',f"top_{top_XX_mechanic}_mechanic"]]

## Create Dataframes for main and subdomain

In [13]:
df_main = get_dataframe(f"SELECT * FROM {schema}.{main}")
df_sub = get_dataframe(f"SELECT * FROM {schema}.{subdomain}")
df_u_sub = get_dataframe(f"SELECT * FROM {schema}.{unique_subdomain}")

- merge Kickstarter with subdomains => 1021 non-null

In [14]:
df_sub = pd.merge(df_sub,df_u_sub,on="subdomain_id")
df_ks_sub = pd.merge(df_ks,df_sub,on='id')

- merge Kickstarter_subdomains with main

In [15]:
df_ks_sub_main = pd.merge(df_ks_sub,df_main,on='id')

- Build new columns with goal in USD

In [16]:
df_ks_sub_main["usd_goal"] = df_ks_sub_main['goal']*(df_ks_sub_main.pledged/df_ks_sub_main.usd_pledged)

# Machine Learning
- Extract only necessary columns for our ML and drop all null

In [17]:
df_ML = df_ks_sub_main[['country','usd_pledged','subdomain_name','min_players','max_players','min_playtime','max_playtime','min_age','averageweight','usd_goal']]
#df_ML.dropna(inplace=True);

In [18]:
df_ML["min_playtime"].fillna(0,inplace=True);
df_ML["max_playtime"].fillna(df_ML["max_playtime"].max(),inplace=True);

/var/folders/4q/cl23pkln7hj7tnvr0yw_yhkw0000gn/T/ipykernel_8378/2015787152.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML["min_playtime"].fillna(0,inplace=True);
/var/folders/4q/cl23pkln7hj7tnvr0yw_yhkw0000gn/T/ipykernel_8378/2015787152.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ML["max_playtime"].fillna(df_ML["max_playtime"].max(),inplace=True);


In [19]:
df_ML = df_ML[(np.abs(stats.zscore(df_ML['min_playtime'])) < 2.5)]
df_ML = df_ML[(np.abs(stats.zscore(df_ML['max_playtime'])) < 3)]

In [20]:
df_ML.describe()

,usd_pledged,min_players,max_players,min_playtime,max_playtime,min_age,averageweight,usd_goal
count,3.190000e+02,319.000000,319.000000,319.000000,319.000000,319.000000,319.000000,3.190000e+02
mean,1.373996e+05,1.786834,4.711599,52.047022,88.573668,10.880878,2.376850,3.689098e+04
std,2.770440e+05,0.717054,5.881820,36.361752,74.854702,3.264770,0.776606,2.024870e+05
min,4.049000e+01,1.000000,1.000000,5.000000,10.000000,0.000000,1.000000,4.569029e+00
25%,1.852900e+04,1.000000,2.000000,30.000000,40.000000,10.000000,1.900000,5.000000e+03
50%,4.366801e+04,2.000000,4.000000,45.000000,60.000000,12.000000,2.300000,1.240045e+04
75%,1.037640e+05,2.000000,6.000000,60.000000,120.000000,13.000000,2.857200,2.500000e+04
max,2.559458e+06,7.000000,100.000000,180.000000,480.000000,17.000000,4.616300,2.513552e+06


In [21]:
df_ML.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 319 entries, 0 to 335
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         319 non-null    object 
 1   usd_pledged     319 non-null    float64
 2   subdomain_name  319 non-null    object 
 3   min_players     319 non-null    float64
 4   max_players     319 non-null    float64
 5   min_playtime    319 non-null    float64
 6   max_playtime    319 non-null    float64
 7   min_age         319 non-null    int64  
 8   averageweight   319 non-null    float64
 9   usd_goal        319 non-null    float64
dtypes: float64(7), int64(1), object(2)
memory usage: 27.4+ KB


In [22]:
subdomain_dummy = pd.get_dummies(df_ML.subdomain_name.apply(pd.Series).stack(), drop_first=True).sum(level=0)
subdomain_dummy.columns = subdomain_dummy.columns.str.strip()

/var/folders/4q/cl23pkln7hj7tnvr0yw_yhkw0000gn/T/ipykernel_8378/919643619.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  subdomain_dummy = pd.get_dummies(df_ML.subdomain_name.apply(pd.Series).stack(), drop_first=True).sum(level=0)


In [23]:
country_dummy = pd.get_dummies(df_ML.country.apply(pd.Series).stack(), drop_first=True).sum(level=0)
country_dummy.columns = country_dummy.columns.str.strip()

/var/folders/4q/cl23pkln7hj7tnvr0yw_yhkw0000gn/T/ipykernel_8378/3043570070.py:1: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  country_dummy = pd.get_dummies(df_ML.country.apply(pd.Series).stack(), drop_first=True).sum(level=0)


In [24]:
df = pd.concat([df_ML,subdomain_dummy], axis=1)
df = df.drop(["subdomain_name"], axis=1)
df.columns

Index(['country', 'usd_pledged', 'min_players', 'max_players', 'min_playtime',
       'max_playtime', 'min_age', 'averageweight', 'usd_goal', 'Children's',
       'Customizable', 'Family', 'Party', 'Strategy', 'Thematic', 'Wargames'],
      dtype='object')

In [25]:
df = pd.concat([df,country_dummy], axis=1)
df = df.drop(["country"], axis=1)
df.columns

Index(['usd_pledged', 'min_players', 'max_players', 'min_playtime',
       'max_playtime', 'min_age', 'averageweight', 'usd_goal', 'Children's',
       'Customizable', 'Family', 'Party', 'Strategy', 'Thematic', 'Wargames',
       'AU', 'BE', 'CA', 'CH', 'DE', 'ES', 'FR', 'GB', 'HK', 'IT', 'NL', 'NO',
       'SG', 'US'],
      dtype='object')

In [26]:
X = df[['AU', 'BE', 'CA', 'CH', 'DE', 'ES', 'FR', 'GB', 'HK', 'IT', 'NL', 'NO', 'SG', 'US',
        "Children's", "Customizable", "Family", "Party", "Strategy", "Thematic", "Wargames", 
        "min_players", "max_players", "min_playtime", "min_age", "averageweight", 'usd_goal']]

In [27]:
y = df.usd_pledged

In [36]:
df2 = df[['DE', "Party", "min_playtime", "averageweight", 'usd_goal', "usd_pledged"]]

In [28]:
X = sm.add_constant(X)

In [29]:
# create an OLS model
our_model = sm.OLS(y, X)

# use the data to calculate the intercept and slope
model_results = our_model.fit()

# return the output of the model
model_results.summary() # summary contains eg. 'const' (intercept) and 'slope' of the regression equation.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            usd_pledged   R-squared:                       0.222
Model:                            OLS   Adj. R-squared:                  0.150
Method:                 Least Squares   F-statistic:                     3.084
Date:                Wed, 13 Jul 2022   Prob (F-statistic):           1.36e-06
Time:                        12:01:12   Log-Likelihood:                -4409.7
No. Observations:                 319   AIC:                             8875.
Df Residuals:                     291   BIC:                             8981.
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -9.687e+04   2.06e+05     -0.471      0.638   -5.01e+05    3.08e+05
AU             5.406e+04   2.14e+05      0.252      0.801   -3.67e+05    4.75e+05
BE             2.713e+05   2.24e+05      1.211      0.227    -1.7e+05    7.12e+05
CA             2.382e+05   2.07e+05      1.150      0.251   -1.69e+05    6.46e+05
CH            -9731.5544   2.37e+05     -0.041      0.967   -4.75e+05    4.56e+05
DE             4.584e+05   2.26e+05      2.027      0.044    1.33e+04    9.04e+05
ES             5.702e+04   2.18e+05      0.262      0.794   -3.72e+05    4.86e+05
FR             5.663e+04   2.03e+05      0.280      0.780   -3.42e+05    4.55e+05
GB             9.539e+04    1.9e+05      0.503      0.615   -2.78e+05    4.69e+05
HK            -1.101e+05   2.71e+05     -0.407      0.684   -6.43e+05    4.23e+05
IT             1.092e+05   2.06e+05      0.529      0.597   -2.97e+05    5.16e+05
NL             1.082e+05   2.23e+05      0.485      0.628   -3.31e+05    5.47e+05
NO            -2.786e+05   3.36e+05     -0.829      0.408    -9.4e+05    3.83e+05
SG            -3.274e+04   3.18e+05     -0.103      0.918   -6.58e+05    5.93e+05
US             1.143e+05   1.86e+05      0.615      0.539   -2.51e+05     4.8e+05
Children's    -6.694e+04   1.14e+05     -0.589      0.556   -2.91e+05    1.57e+05
Customizable  -7.188e+04   1.89e+05     -0.381      0.704   -4.43e+05       3e+05
Family         2.601e+04   6.24e+04      0.417      0.677   -9.67e+04    1.49e+05
Party          4.373e+05   8.52e+04      5.130      0.000     2.7e+05    6.05e+05
Strategy      -1.016e+04   6.38e+04     -0.159      0.874   -1.36e+05    1.15e+05
Thematic       6.225e+04   6.71e+04      0.927      0.355   -6.99e+04    1.94e+05
Wargames      -6.535e+04   6.56e+04     -0.996      0.320   -1.94e+05    6.38e+04
min_players   -2.684e+04   2.28e+04     -1.176      0.241   -7.18e+04    1.81e+04
max_players   -2015.7265   2604.264     -0.774      0.440   -7141.307    3109.854
min_playtime   -715.9943    528.931     -1.354      0.177   -1757.010     325.021
min_age       -4932.1560   5258.184     -0.938      0.349   -1.53e+04    5416.736
averageweight  1.028e+05   2.74e+04      3.748      0.000    4.88e+04    1.57e+05
usd_goal          0.3250      0.138      2.361      0.019       0.054       0.596
==============================================================================
Omnibus:                      282.973   Durbin-Watson:                   1.555
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6835.612
Skew:                           3.601   Prob(JB):                         0.00
Kurtosis:                      24.504   Cond. No.                     1.05e+07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of th

In [54]:
X = df[['averageweight','usd_goal','min_playtime','Party','DE']]
X = sm.add_constant(X)
X.describe()

,const,averageweight,usd_goal,min_playtime,Party,DE
count,319.0,319.000000,3.190000e+02,319.000000,319.000000,319.000000
mean,1.0,2.376850,3.689098e+04,52.047022,0.053292,0.012539
std,0.0,0.776606,2.024870e+05,36.361752,0.224967,0.111449
min,1.0,1.000000,4.569029e+00,5.000000,0.000000,0.000000
25%,1.0,1.900000,5.000000e+03,30.000000,0.000000,0.000000
50%,1.0,2.300000,1.240045e+04,45.000000,0.000000,0.000000
75%,1.0,2.857200,2.500000e+04,60.000000,0.000000,0.000000
max,1.0,4.616300,2.513552e+06,180.000000,1.000000,1.000000


In [40]:
df2 = sm.add_constant(df2)

In [49]:
col = df2.columns.to_list()
col

['const',
 'DE',
 'Party',
 'min_playtime',
 'averageweight',
 'usd_goal',
 'usd_pledged']

In [46]:
const = [df2["const"].to_list()]
avgwei = [df2["averageweight"].to_list()]
usd_goal = [df2["usd_goal"].to_list()]
min_playtime = [df2["min_playtime"].to_list()]
party = [df2["Party"].to_list()]
DE = [df2["DE"].to_list()]
usd_pledged = [df2["usd_pledged"].to_list()]

const

## Für Christopher


[[1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,

In [50]:
dic = {
col[0]: const,
col[1]: DE,
col[2]: party,
col[3]: min_playtime,
col[4]: avgwei,
col[5]: usd_goal,
col[6]: usd_pledged
}

In [51]:
df_lin_re = pd.DataFrame(dic)
df_lin_re

,const,DE,Party,min_playtime,averageweight,usd_goal,usd_pledged
0,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[90.0, 15.0, 120.0, 5.0, 60.0, 10.0, 60.0, 30....","[2.625, 1.0385, 2.6667, 1.675, 2.2534, 2.0714,...","[786.7978359249521, 44415.55900059686, 27663.0...","[10105.01, 210531.26, 63244.0, 12058.0, 76254...."


In [52]:
df_lin_re.to_csv("/Users/jannikduda/neuefische/Daily-Lama-Capstone/data/bgg_lin_re.csv")

In [56]:
# create an OLS model
our_model = sm.OLS(y, X)
#our_model = sm.MANOVA(y, X)

# use the data to calculate the intercept and slope
model_results = our_model.fit()

# return the output of the model
model_results.summary() # summary contains eg. 'const' (intercept) and 'slope' of the regression equation.

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            usd_pledged   R-squared:                       0.173
Model:                            OLS   Adj. R-squared:                  0.160
Method:                 Least Squares   F-statistic:                     13.13
Date:                Wed, 13 Jul 2022   Prob (F-statistic):           1.28e-11
Time:                        14:58:04   Log-Likelihood:                -4419.5
No. Observations:                 319   AIC:                             8851.
Df Residuals:                     313   BIC:                             8873.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const         -8.058e+04   4.92e+04     -1.637      0.103   -1.77e+05    1.63e+04
averageweight  1.052e+05   2.24e+04      4.693      0.000    6.11e+04    1.49e+05
usd_goal          0.1524      0.072      2.120      0.035       0.011       0.294
min_playtime  -1208.8459    459.565     -2.630      0.009   -2113.073    -304.619
Party          3.985e+05   6.61e+04      6.025      0.000    2.68e+05    5.29e+05
DE             3.158e+05   1.28e+05      2.464      0.014    6.36e+04    5.68e+05
==============================================================================
Omnibus:                      278.830   Durbin-Watson:                   1.526
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6227.891
Skew:                           3.555   Prob(JB):                         0.00
Kurtosis:                      23.445   Cond. No.                     1.85e+06
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+06. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [63]:
model_results.rsquared

0.17340417146148845

In [60]:
regr = linear_model.LinearRegression()
regr.fit(X, y)

LinearRegression()

In [61]:
# X = df[['averageweight','usd_goal','min_playtime','Party', 'DE']]
predicted_pledged_amount = regr.predict([[4, 50000, 20, 1, 0]])
predicted_pledged_amount

/opt/homebrew/Caskroom/miniconda/base/envs/nf_sql/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


ValueError: X has 5 features, but LinearRegression is expecting 6 features as input.